# Projeto Final - Otimização e Computação Evolucionária


## Alunos: Eduardo Ximenes, Matheus Gomes Maranhão, Vitor Peter

O presente projeto foi realizado como atividade de conclusão da disciplina de Otimização e Computação Evolucionária. A atividade consiste no uso dos conhecimentos adquiridos ao longo das aulas, através da aplicação do Algoritmos Genético e Particle Swarm Optimization com o objetivo de realizar a clusterização de um dataset e regressão com seleção de features no outro dataset. 

In [21]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math

from sklearn.model_selection import train_test_split
from sklearn.metrics import silhouette_score, jaccard_score, r2_score, mean_squared_error
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.svm import SVR

## Dataset 1: Churn Prediction  
Com este dataset, será feita uma clusterização com o modelo KMeans usando um método fit ajustado por computação evolucionária. A ideia será comparar essa solução com o modelo padrão. A métrica utilizada foi o índice de silhueta.

In [2]:
df_churn = pd.read_csv('./data/churn_prediction.csv')

In [3]:
df_churn

,customer_id,credit_score,country,gender,age,tenure,balance,products_number,credit_card,active_member,estimated_salary,churn
0,15634602,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,15647311,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,15619304,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,15701354,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,15737888,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...,...
9995,15606229,771,France,Male,39,5,0.00,2,1,0,96270.64,0
9996,15569892,516,France,Male,35,10,57369.61,1,1,1,101699.77,0
9997,15584532,709,France,Female,36,7,0.00,1,0,1,42085.58,1
9998,15682355,772,Germany,Male,42,3,75075.31,2,1,0,92888.52,1


## Dataset 2: Housing Prices  
Com este dataset, será feita uma regressão com o modelo SVR usando features selecionadas por computação evolucionária. A ideia será comparar essa solução com o modelo utilizando todas as features, e com o modelo utilizando features escolhidas pelo PCA. As métricas utilizadas foram R² e MSE.

In [64]:
df_housing = pd.read_csv('./data/housing_price.csv')
print(df_housing.head(1))
df_housing.info()

      price  area  bedrooms  bathrooms  stories mainroad guestroom basement  \
0  13300000  7420         4          2        3      yes        no       no   

  hotwaterheating airconditioning  parking prefarea furnishingstatus  
0              no             yes        2      yes        furnished  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 545 entries, 0 to 544
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   price             545 non-null    int64 
 1   area              545 non-null    int64 
 2   bedrooms          545 non-null    int64 
 3   bathrooms         545 non-null    int64 
 4   stories           545 non-null    int64 
 5   mainroad          545 non-null    object
 6   guestroom         545 non-null    object
 7   basement          545 non-null    object
 8   hotwaterheating   545 non-null    object
 9   airconditioning   545 non-null    object
 10  parking           545 non-null    int

In [65]:
dict_house_bool = {'no':0, 'yes':1}
booleans_house = ['mainroad', 'guestroom', 'basement', 'hotwaterheating', 'airconditioning', 'prefarea']
for boolean_col in booleans_house:
    df_housing[boolean_col] = df_housing[boolean_col].map(dict_house_bool)

dict_house_furnish = {False:0, True:1}
dummies_furnish = pd.get_dummies(df_housing.furnishingstatus)
for furnish_col in dummies_furnish:
    dummies_furnish[furnish_col] = dummies_furnish[furnish_col].map(dict_house_furnish)

df_housing = df_housing.drop('furnishingstatus',axis=1)
dummies_furnish = dummies_furnish.drop('unfurnished',axis=1)
for col in dummies_furnish:
    df_housing[col] = dummies_furnish[col]

df_housing.head()

,price,area,bedrooms,bathrooms,stories,mainroad,guestroom,basement,hotwaterheating,airconditioning,parking,prefarea,furnished,semi-furnished
0,13300000,7420,4,2,3,1,0,0,0,1,2,1,1,0
1,12250000,8960,4,4,4,1,0,0,0,1,3,0,1,0
2,12250000,9960,3,2,2,1,0,1,0,0,2,1,0,1
3,12215000,7500,4,2,2,1,0,1,0,1,3,1,1,0
4,11410000,7420,4,1,2,1,1,1,0,1,2,0,1,0


### Computação evolucionária

In [7]:
# Função para calcular a fitness no GA e no PSO
def fitness_function(ind_encoded, beta, X, y):

    # ind é um subconjunto de features: [0 0 1 0 0 1 ...]
    ind = decode_ind(ind_encoded, beta)
    m, n = X.shape
    # inicializar o alpha
    alpha = np.inf
    for i in range(m): # iterate over all samples
        for j in range(i + 1, m): # iterate over all samples
            if y[i] != y[j]: # if i and j are not from the same class
                distance_ij = np.sum([(X[i,k] - X[j,k])**2 * ind[k] for k in range(n)])
                alpha = min(alpha, distance_ij)

    # Maximizando o mínimo alpha, então deve-se negativar o valor antes
    return -alpha

#### Generative Algorithm (GA)

In [8]:
# função que gera pop inicial
def init_pop(P,X):
    """
    Inicialize um conjunto de indivíduos, cada um contendo elementos entre 0 e 1.

    Parâmetros:
    X: dados de treinamento
    P: número de indivíduos na população

    Retorna:
    pop: população inicial
    """
    pop = []
    m, n = X.shape
    for _ in range(P):
        ind = [np.random.uniform(0, 1) for __ in range(n)] # chaves aleatórias
        pop.append(ind)

    return pop

In [9]:
# Função para decodificar o indivíduo
def decode_ind(ind, beta):
    """
    Decodificar um indivíduo definindo 'beta' elementos com a maior
    probabilidade como 1 e os demais como 0.

    Parâmetros:
    ind: indivíduo a ser decodificado.
    beta: O número de elementos a serem definidos como 1.

    Retorna:
    ind: indivíduo a ser decodificado.
    """

    # Obtenha os índices dos 'beta' maiores elementos
    indices_of_highest = sorted(range(len(ind)), key=lambda i: ind[i], reverse=True)[:beta]
    # []

    # Innicialize o indivíduo decodificado
    decoded_ind = [0] * len(ind)

    # Inclua um total de 'beta' elementos iguais a 1
    for index in indices_of_highest:
        decoded_ind[index] = 1

    return decoded_ind

In [10]:
def select_individuals(population, elite_size): # elitism is used
    """
    Selecionar dois indivíduos aleatoriamente da população P vezes.
    Como utilizamos elitismo, devemos reduzir o número de indivídos selecionados

    Parâmetros:
    population: Lista ou array representando a população.
    elite_size: Tamanho da elite

    Retorna:
    Lista de tuplas, cada uma contendo os índices de dois indivíduos selecionados aleatoriamente.
    """
    population_size = len(population)-math.ceil(elite_size)

    selections = []
    for _ in range(population_size):
        # Selecione dois indivíduos aleatoriamente com reposição
        selected_indices = np.random.choice(population_size, size=2, replace=True)
        selections.append(tuple(selected_indices))

    return selections

In [11]:
def uniform_crossover(parent1, parent2, crossover_rate=0.5):
    """
    Realizar crossover uniforme entre dois pais.

    Parâmetros:

    parent1: genes do primeiro pai.
    parent2: genes do segundo pai.
    crossover_rate: Probabilidade de selecionar um gene do primeiro pai.

    Retorna:
    genes do filho após o crossover uniforme.
    """

    # Gerar uma lista aleatória booleana, onde o elemento da lista é igual a True, se p < crossover_rate
    mask = np.random.rand(len(parent1)) < crossover_rate

    # Realizar crossover
    child = np.where(mask, parent1, parent2)

    return child.tolist()

In [12]:
def mutation(child, p_m=0.2, mutation_std_dev=0.01):
    """

    Aplicar mutação aos genes de uma criança.

    Parâmetros:
    child: genes da criança.
    p_m: Probabilidade de aplicar mutação a cada gene.
    mutation_std_dev: Desvio padrão da distribuição normal para a mutação.

    Retorna:
    genes da criança após a mutação.

    * obs.: o std_dev foi reduzido e 0.1 para 0.01
    """
    mutated_child = np.copy(child)

    # Faça o loop em cada gene e mute-o com probabilidade p_m
    for i in range(len(child)):
        if np.random.rand() < p_m:
            # Aplique a mutação perturbando o gene com uma distribuição normal
            mutated_child[i] += np.random.normal(loc=0, scale=mutation_std_dev)

    return mutated_child.tolist()

In [13]:
def genetic_algorithm(pop_size, p_crossover, p_mutation, data, labels, beta, max_generations, elite_size):
    """
    Loop do GA.

    Parâmetros:
    - pop_size: Population size.
    - p_crossover: Crossover probability.
    - p_mutation: Mutation probability.
    - data: Dataset.
    - labels: labels das amostras.
    - beta: tamanho do subconjunto de features
    - max_generations: Maximum number of generations.
    - elite_size: tamanho da elite

    Retorna:
    - Melhor subconjunto de características obtido pelo GA
    """
    # inicializar os dados
    X = data
    y = labels
    # gerar pop inicial
    pop = init_pop(pop_size,X)
    # Listas para guardar melhores fitness e indivíduos
    fitness_over_time = []
    bestind_over_time = []
    for generation in range(max_generations):
        print("G: " + str(generation))
        # Avaliar a fitness de cada indivíduo
        #fitness_values = [fitness_function(ind, beta, X, y) for ind in pop]
        # Encontrar melhores indivíduos de acordo com a fitness
        best_ind = sorted(pop, key=lambda ind: fitness_function(ind, beta, X, y))[:math.ceil(elite_size)]
        # Guardar as melhores fitness e indivíduos
        best_fitness_val = fitness_function(best_ind[0], beta, X, y)
        print(-best_fitness_val)
        fitness_over_time.append(-best_fitness_val)
        bestind_over_time.append(best_ind[0])
        #print(best_ind[0])

        # Selecionar pais para reprodução
        selections = select_individuals(pop, elite_size)

        # Gerar nova pop através de crossover e mutação
        offspring = []
        for parent1, parent2 in selections:
            child = uniform_crossover(pop[parent1], pop[parent2], p_crossover)
            child = mutation(child, p_mutation)
            offspring.append(child)

        # Atualizar população
        pop = offspring
        pop.extend(best_ind)

    return best_ind[0], fitness_over_time, bestind_over_time

#### Particle Swarm Optimization (PSO)

In [14]:
# função que gera enxame inicial
def init_swarm(P,X):
    """
    Inicialize um enxame com P partículas, cada uma contendo elementos entre 0 e 1.

    Parâmetros:
    X: dados de treinamento
    P: número de indivíduos no enxame

    Retorna:
    pos = posição inicial das P partículas do enxame
    vel = velocidade inicial das P partículas do enxame
    pos_best = vetor de melhores posições conhecidas pela partícula
    """
    pos = []
    vel = []
    pos_best = []
    m, n = X.shape
    for _ in range(P):
        pos_ind = [np.random.uniform(0, 1) for __ in range(n)] # chaves aleatórias
        pos.append(pos_ind)
        pos_best.append(pos_ind)
        vel.append(np.random.uniform(-1,1, size = n))

    return pos, vel, pos_best

In [15]:
# atualizar velocidade da partícula
def update_velocity(pos, vel, pos_best, global_best, w, c1, c2):
  '''
    Atualiza a velocidade da partícula usando a fórmula da Otimização por Enxame de Partículas (PSO).

    Parâmetros:
    pos: posição atual da partícula
    vel: velocidade atual da partícula
    pos_best: Melhor posição global entre todas as partículas.
    c1: Parâmetro cognitivo.
    c2: Parâmetro social.
    w: Peso de inércia.

    Retorna:
    - Velocidade atualizada para a partícula.
  '''
  # inicializar vel atualizada
  vel_updated = []
  # loop em cada posição i da partícula
  for i in range(len(pos)):
      # Gera números aleatórios no intervalo [0, 1]
      r1 = np.random.uniform(0,1)
      r2 = np.random.uniform(0,1)

      # Calcula os componentes cognitivo e social da atualização de velocidade
      vel_cognitive = c1 * r1 * (pos_best[i] - pos[i])
      vel_social = c2 * r2 * (global_best[i] - pos[i])

      # Atualiza a velocidade i usando a fórmula do PSO
      vel_updated.append(w * vel[i] + vel_cognitive + vel_social)

  # Retorna a velocidade atualizada para a partícula
  return np.array(vel_updated)

In [16]:
# definir função de atualização da posição
def update_position(pos, vel, bounds):
  '''
    Atualiza a posição da partícula a partir da nova velocidade.

    Parâmetros:
    pos: posição atual da partícula
    vel: velocidade atualizada da partícula
    bounds: limites do ambiente

    Retorna:
    - Velocidade atualizada para a partícula.
  '''
  pos_updated = []
  for i in range(len(pos)):
      pos_updated.append(np.clip(pos[i]+vel[i], a_min=bounds[0], a_max=bounds[1]))
  return pos_updated

In [17]:
def pso(swarm_size, c1, c2, w, data, labels, beta, max_iters):
  """
  Loop do PSO.

  Parâmetros:
  swarm_size: num de partículas.
  c1: Parâmetro cognitivo.
  c2: Parâmetro social.
  w: Peso de inércia.
  data: Dataset.
  labels: labels das amostras.
  beta: tamanho do subconjunto de features
  max_iters: Número máximo de períodos.

  Retorna:
  Melhor subconjunto de features obtido pelo pso
  """
  # inicializar os dados
  X = data
  y = labels
  bounds = [0, 1] # já que posições são chaves aleatórias
  # inicializar o enxame
  pos_swarm, vel_swarm, pos_best_swarm = init_swarm(swarm_size,X)
  # lista para armazenar evolução da fitness
  fitness_over_time = []
  bestind_over_time = []

  # comece o loop
  best_ind = pos_swarm[0] # initialize global best
  for iter in range(max_iters):
      print("iter: " + str(iter))
      # avaliar fitness das partículas do enxame
      global_best_fitness = fitness_function(best_ind, beta, X, y) # fitness da melhor posição global
      for j,ind in enumerate(pos_swarm):
        score = fitness_function(ind, beta, X, y) # fitness da posição atual
        score_known_best = fitness_function(pos_best_swarm[j], beta, X, y) # fitness da melhor posição conhecida
        if score < score_known_best:
          pos_best_swarm[j] = ind # se uma posição melhor for encontrada
        if score < global_best_fitness:
          best_ind = ind

      # armazenar melhor fitness deste período
      best_fitness_val = fitness_function(best_ind, beta, X, y)
      print(-best_fitness_val)
      fitness_over_time.append(-best_fitness_val)
      bestind_over_time.append(best_ind)

      # cycle through swarm and update velocities and position
      for j in range(swarm_size):
          vel_swarm[j] = update_velocity(pos_swarm[j], vel_swarm[j], pos_best_swarm[j], best_ind, w, c1, c2)
          #pos_swarm[j] = update_position(pos_swarm[j], vel_swarm[j], [-1000,1000])
          pos_swarm[j] = update_position(pos_swarm[j], vel_swarm[j], bounds)

  return best_ind, fitness_over_time, bestind_over_time

### Aplicação no modelo

In [67]:
# Dividindo o conjunto de dados em treino e teste
X_house = df_housing.drop('price',axis=1).copy()
y_house = df_housing.price.copy()
X_train_house, X_test_house, y_train_house, y_test_house = train_test_split(X_house, y_house, test_size=0.25, random_state=101)

#### GA

#### PSO

#### PCA

In [68]:
pca_house = PCA(n_components=int(X_train_house.shape[1]/2))
X_train_house_PCA = pca_house.fit_transform(X_train_house)
X_test_house_PCA = pca_house.fit_transform(X_test_house)

#### Modelo padrão

#### Discussão dos resultados  
O modelo com as melhores métricas foi tal, com R² de X e MSE de Y.